# Clean Data

Historical odds and results data sourced from [here](https://www.sportsbookreviewsonline.com/scoresoddsarchives/nba/nbaoddsarchives.htm).

In [171]:
import datetime
import hashlib
import os
import pandas as pd

In [311]:
def parse(game_tuple):
    row = {
        'raw_date': game_tuple[0]['Date'],
        'team': game_tuple[0]['Team'],
        'VH': game_tuple[0]['VH'],
        'team_points': game_tuple[0]['Final'],
        'opponent': game_tuple[1]['Team'],
        'opponent_points': game_tuple[1]['Final'],
        'point_spread': get_point_spread(game_tuple),
    }

    return row

def get_game_spread(game_tuple):
    '''
    Data model here assigns the point spread to the favorite and the game total to the dog. 
    
    We need to untangle this unholy mess.
    '''
    game_spread = [clean_close(x['Close']) for x in game_tuple if clean_close(x['Close']) < 100][0]

    return game_spread

def get_point_spread(game_tuple):
    game_spread = get_game_spread(game_tuple)
    
    if game_spread == game_tuple[0]['Close']:
        return -1 * game_spread
    else:
        return game_spread
    
def clean_close(close):
    if close in ['pk', 'PK']:
        return 0
    elif close != close:
        return 0
    else:
        return close
    
def get_game_tuples(file_name):
    source_data = pd.read_excel('../data/sportsbookreview_nba/' + file_name)
    records = source_data.to_dict('records')
    game_tuples = [records[i:i+2] for i in range(0, len(records), 2)]
    
    return game_tuples

def get_game_date(row):
    raw_date = row['raw_date']
    
    year = int(row['season'].split('-')[0])
    if raw_date < 800:
        year+=1
        
    month = int(str(raw_date)[:-2])
    day = int(str(raw_date)[-2:])
    
    dt = datetime.date(year, month, day)
    
    return dt

def get_clean_name(input_string):
    if input_string == "Oklahoma City":
        return 'OklahomaCity'
    elif input_string == "LA Clippers":
        return 'LAClippers'
    elif input_string == "Golden State":
        return 'GoldenState'
    elif input_string in ['NewJersey', 'Brooklyn']:
        return 'Nets'
    else:
        return input_string

In [280]:
def get_game_id(row):
    raw_id = row['team'] + row['opponent'] + row['file_name'] + str(row['raw_date'])
    m = hashlib.md5()
    m.update(raw_id.encode())
    game_id = m.hexdigest()
    
    return game_id

In [289]:
files = os.listdir('../data/sportsbookreview_nba')
files

['nba odds 2017-18.xlsx',
 'nba odds 2009-10.xlsx',
 'nba odds 2021-22.xlsx',
 'nba odds 2010-11.xlsx',
 'nba odds 2019-20.xlsx',
 'nba odds 2007-08.xlsx',
 'nba odds 2014-15.xlsx',
 'nba odds 2022-23.xlsx',
 'nba odds 2018-19.xlsx',
 'nba odds 2012-13.xlsx',
 'nba odds 2015-16.xlsx',
 '.ipynb_checkpoints',
 'nba odds 2016-17.xlsx',
 'nba odds 2008-09.xlsx',
 'nba odds 2020-21.xlsx',
 'nba odds 2013-14.xlsx',
 'nba odds 2011-12.xlsx']

In [290]:
rows = []

for file_name in files:
    if 'xlsx' in file_name:
        print(file_name)

        game_tuples = get_game_tuples(file_name)

        for game_tuple in game_tuples:
            ## Get First Team
            row = parse(game_tuple)
            row.update({'file_name': file_name})
            game_id = get_game_id(row)
            row.update({'game_id': game_id})
            rows.append(row)

            ## Now, Reverse Order and Get Second Team
            game_tuple.reverse()

            row = parse(game_tuple)
            row.update({'file_name': file_name})
            row.update({'game_id': game_id})
            rows.append(row)

nba odds 2017-18.xlsx
nba odds 2009-10.xlsx
nba odds 2021-22.xlsx
nba odds 2010-11.xlsx
nba odds 2019-20.xlsx
nba odds 2007-08.xlsx
nba odds 2014-15.xlsx
nba odds 2022-23.xlsx
nba odds 2018-19.xlsx
nba odds 2012-13.xlsx
nba odds 2015-16.xlsx
nba odds 2016-17.xlsx
nba odds 2008-09.xlsx
nba odds 2020-21.xlsx
nba odds 2013-14.xlsx
nba odds 2011-12.xlsx


In [312]:
clean_data = (
    pd.DataFrame(rows)
    .assign(
        season = lambda x: x['file_name'].transform(lambda s: s.split(' ')[-1].split('.')[0]),
        dt = lambda x: x.apply(get_game_date, axis=1),
        home_away = lambda x: x['VH'].transform(lambda s: 'Home' if s == "H" else "Away"),
        mov = lambda x: x['team_points'] - x['opponent_points'],
        team = lambda x: x['team'].apply(get_clean_name),
        opponent = lambda x: x['opponent'].apply(get_clean_name)
    )
    .sort_values(
        by=['season', 'team', 'dt']
    )
    .reset_index()
    .reset_index()
    .assign(
        game_number = lambda x: x.groupby(['season', 'team'])['level_0'].rank(method="first", ascending=True)
    )
    .drop(['raw_date', 'VH', 'level_0', 'index'], axis=1)
)

clean_data.sample(10)

,team,team_points,opponent,opponent_points,point_spread,file_name,game_id,season,dt,home_away,mov,game_number
195,Boston,98,LALakers,103,7.5,nba odds 2007-08.xlsx,c89917dd67179614d9a63902072b1391,2007-08,2008-06-15,Away,-5,107.0
29667,Miami,115,Orlando,91,2.5,nba odds 2018-19.xlsx,a5990dc43a7e55b0c225bef4eba73643,2018-19,2018-12-23,Away,24,32.0
26929,LALakers,127,Cleveland,113,2.0,nba odds 2017-18.xlsx,dad8a07ffb563695b138204d2c71ad45,2017-18,2018-03-11,Home,14,66.0
565,Denver,91,SanAntonio,102,6.0,nba odds 2007-08.xlsx,cb1a6a81f5574dca645c36478c87cc54,2007-08,2007-12-15,Away,-11,23.0
16234,Indiana,93,Washington,80,4.5,nba odds 2013-14.xlsx,f283d4bbc6d268894682e097eb12de17,2013-14,2014-05-15,Away,13,95.0
6484,Memphis,108,Denver,125,1.5,nba odds 2009-10.xlsx,f4b6f4942dea3d451629d126dcc68d4d,2009-10,2010-03-13,Home,-17,67.0
33996,GoldenState,122,Milwaukee,121,3.0,nba odds 2020-21.xlsx,afbaff02fe80fb17942fd3456415413b,2020-21,2021-04-06,Home,1,51.0
6034,GoldenState,117,Dallas,127,5.0,nba odds 2009-10.xlsx,08d153cb40835cae72ff6f57941db554,2009-10,2010-02-08,Home,-10,50.0
38312,Washington,132,Minnesota,114,12.0,nba odds 2021-22.xlsx,6b77e4a4b9a285fbe4d09c0c7d4f70d0,2021-22,2022-04-05,Away,18,79.0
25951,Boston,110,Toronto,99,4.5,nba odds 2017-18.xlsx,0b38d4733a1400472518d3bef68c9471,2017-18,2018-03-31,Home,11,76.0


In [313]:
clean_data.to_csv('../data/NBA Scores and Point Spreads.csv', index=False)